In [1]:
import sqlalchemy
from bs4 import BeautifulSoup
import requests
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_date
from pyspark.sql.functions import *
import pandas as pd
from pyspark.sql.types import *
from sqlalchemy import create_engine
import os
import json
import time

In [2]:
# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Soccer Data") \
    .getOrCreate()

In [8]:
# def fetch_matchday(self):
log_name = "matchday"
url = f"https://www.sofascore.com/api/v1/sport/football/scheduled-events/2024-05-09"
response = requests.get(url)
status_code = response.status_code

if status_code != 200:
    print(f"Error code: {status_code}. Stop jobs!")
else:
    match_in_day = response.json()
    match_in_day = match_in_day['events']

    # Normalize the nested JSON data
    df_matchday = pd.json_normalize(match_in_day)
    df_matchday.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
    df_matchday.rename(columns={"id": "match_id"}, inplace=True)
    df = df_matchday[df_matchday['tournament_category_sport_name'] == 'Football']
    print("Dataframe is here!!!")
    print(df)
    list_match_id = df["match_id"].astype(str).tolist()
    # self.export(log_name, df)
    # Fetch and export shotmap data
    # shotmap_df = shotmap(list_match_id)
    # if shotmap_df is not None:
    #     self.export("shotmap", shotmap_df)

# def shotmap(self, list_match_id):
#     shotmap_data = []
# 
#     for match_id in list_match_id:
#         url = f"https://www.sofascore.com/api/v1/event/{match_id}/shotmap"
#         try:
#             response = requests.get(url)
#             status_code = response.status_code
#             if status_code == 200:
#                 data = response.json()
#                 for event in data:
#                     event['match_id'] = match_id  # Add match_id to each event
#                 shotmap_data.extend(data)
#             else:
#                 print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
#         except Exception as e:
#             print(f"An error occurred for match ID {match_id}: {e}")
#         time.sleep(1)
# 
#     if shotmap_data:
#         # Normalize the nested JSON data
#         df_shotmap = pd.json_normalize(shotmap_data)
#         df_shotmap.rename(columns=lambda col: col.replace('.', '_'), inplace=True)
#         return df_shotmap
#     else:
#         return None

Dataframe is here!!!
     customId  winnerCode  aggregatedWinnerCode  hasGlobalHighlights  \
0     xqbsEgb         1.0                   1.0                False   
1      CHsLdb         1.0                   1.0                False   
2     Hdbsceb         3.0                   1.0                 True   
3     TdbsNhy         3.0                   2.0                False   
4       PsVoc         1.0                   1.0                False   
..        ...         ...                   ...                  ...   
173  jIisvQDb         1.0                   NaN                False   
174  MucsBxFb         2.0                   NaN                 True   
175   Fkntvmn         1.0                   NaN                False   
176   AmvsEmn         1.0                   NaN                False   
177  FmnsvCRb         2.0                   NaN                 True   

    hasEventPlayerStatistics hasEventPlayerHeatMap  detailId  \
0                       True                  True

In [9]:
print(list_match_id)

['12173508', '12173471', '12173472', '12173517', '12173516', '12172425', '12172419', '12172525', '12172526', '12172437', '12172434', '12172441', '12172453', '12172509', '12172501', '12172345', '12172348', '12172359', '12172356', '12172350', '12172339', '12172333', '12172328', '12172329', '12214127', '12213731', '12310661', '12310660', '12310659', '12310668', '12255885', '12255883', '12255882', '12255880', '12146236', '11457108', '12310473', '12060302', '12060303', '12060304', '12287635', '12097656', '12097663', '12097660', '12097664', '12004537', '11453188', '11453182', '11453194', '11453178', '12295020', '12295017', '12295018', '12295013', '12091457', '11378835', '12026342', '12026337', '12026344', '11424125', '11451575', '11451642', '11451574', '11451572', '11451573', '11451571', '12277222', '12277221', '12297504', '12297506', '12297496', '12297499', '12288965', '12288963', '12288962', '12288968', '12288964', '12288971', '12162795', '12162793', '12115569', '12264393', '12270392', '12

In [15]:
list_match_id = ["12173508"]
shotmap_data = []

for match_id in list_match_id:
    url = f"https://www.sofascore.com/api/v1/event/"+match_id+"/shotmap"
    try:
        response = requests.get(url)
        status_code = response.status_code
        if status_code == 200:
            data = response.json()
            for event in data['shotmap']:
                event_data = event.copy()  # Create a copy of the event dictionary
                event_data['match_id'] = match_id  # Add match_id to the copy
                shotmap_data.append(event_data)  # Append the modified event data
        else:
            print(f"Failed to fetch data for match ID {match_id}. Status code: {status_code}")
    except Exception as e:
        print(f"An error occurred for match ID {match_id}: {e}")
    time.sleep(1)

if shotmap_data:
    # Normalize the nested JSON data
    df_shotmap = pd.json_normalize(shotmap_data)
    df_shotmap.rename(columns=lambda col: col.replace('.', '_'), inplace=True)

In [16]:
print(df_shotmap)

    isHome shotType   situation    bodyPart goalMouthLocation        xg  \
0     True     miss    assisted   left-foot              left  0.037234   
1    False     goal      corner  right-foot        low-centre  0.843225   
2    False    block     regular  right-foot        low-centre  0.130140   
3    False     goal     regular  right-foot          low-left  0.674115   
4     True     save    assisted  right-foot        low-centre  0.038119   
5     True     miss    assisted  right-foot              high  0.266474   
6    False     miss    assisted   left-foot        close-left  0.050242   
7     True     goal  fast-break  right-foot        high-right  0.060754   
8     True     save    assisted   left-foot         high-left  0.044519   
9    False     miss   set-piece  right-foot        high-right  0.012618   
10    True     save    assisted  right-foot       high-centre  0.051032   
11    True     save   free-kick  right-foot          low-left  0.046737   
12    True     miss    as

In [38]:
# Extract shotmap data
shotmap = shots['shotmap']

# Normalize the nested JSON data
df = pd.json_normalize(shotmap)

# Rename columns
df.rename(columns=lambda col: col.replace('.', '_'), inplace=True)

# Display the DataFrame
print(df)

    isHome shotType   situation    bodyPart goalMouthLocation        xg  \
0    False    block      corner   left-foot          low-left  0.069473   
1    False     goal      corner   left-foot         low-right  0.043971   
2    False     miss    assisted  right-foot              high  0.034340   
3    False    block     regular  right-foot          low-left  0.039987   
4     True     save   set-piece        head         low-right  0.035561   
5     True     goal    assisted  right-foot          low-left  0.049927   
6    False     miss     regular  right-foot             right  0.010895   
7     True     miss      corner        head              left  0.091521   
8     True     miss    assisted   left-foot       close-right  0.197194   
9     True     miss      corner        head              left  0.182517   
10   False     save    assisted        head         high-left  0.099669   
11   False     save    assisted  right-foot        low-centre  0.487510   
12   False     miss      

In [45]:
# Step 2: Create a SQLAlchemy engine
# Replace the placeholders with your PostgreSQL credentials and database details
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'localhost'  # Change to your database endpoint
USER = 'postgres'
PASSWORD = 'ducnm7'
PORT = 5432
DATABASE = 'postgres'

# Connection string
connection_string = f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

# Step 3: Export the DataFrame to a PostgreSQL table
table_name = 'shotmap'
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"DataFrame exported to PostgreSQL table '{table_name}' successfully.")

DataFrame exported to PostgreSQL table 'shotmap' successfully.


In [40]:
url = "https://www.sofascore.com/newcastle-united-manchester-united/DYBe#12240571"
response = requests.get(url)